In [1]:
import pandas as pd
import numpy as np

In [17]:
data = pd.read_csv('games_may2024_cleaned.csv')

In [18]:
data

,AppID,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,29018,753,6078,352,1362469,"{'FPS': 90076, 'Shooter': 64786, 'Multiplayer'...",87,8071426,80,56072
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,590582,"{'Survival': 14611, 'Shooter': 12477, 'Battle ...",58,2365012,68,19517
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,35557,1327,1163,977,668192,"{'Free to Play': 59559, 'MOBA': 19966, 'Multip...",81,2247365,71,23832
3,271590,Grand Theft Auto V,2015-04-13,17,0.00,1,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,12898,590,6910,235,133571,"{'Open World': 31998, 'Action': 23177, 'Multip...",87,1643791,92,16181
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,19.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,16213,578,3016,331,68162,"{'FPS': 9689, 'PvP': 9073, 'e-sports': 9026, '...",85,1091695,77,10566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83641,1745960,Circuit Breaker,2021-10-21,0,4.99,1,Circuit Breaker is a turn-based strategy game ...,Circuit Breaker is a turn-based strategy game ...,"In the cyber-future of 2009, the most popular ...",NaN,...,0,0,0,0,0,"{'Strategy': 72, 'Casual': 48, 'PvE': 47, 'Sin...",-1,-1,-1,-1
83642,1888920,Chess vs Chat,2022-08-09,0,3.99,0,Chess Vs Chat allows streamers to host chess m...,Chess Vs Chat allows streamers to host chess m...,Allow your chatters to actively participate an...,NaN,...,0,0,0,0,0,"{'Indie': 31, 'Strategy': 21, 'Puzzle': 11, 'M...",-1,-1,-1,-1
83643,2166420,Orbtangle,2022-10-28,0,1.99,0,Orbtangle is a simplistic puzzle game which re...,Orbtangle is a simplistic puzzle game which re...,Orbtangle is a simplistic puzzle game which re...,NaN,...,0,0,0,0,0,"{'Puzzle': 54, 'Colorful': 46, 'Relaxing': 44,...",-1,-1,-1,-1
83644,1456260,Flashlight,2021-10-25,0,2.99,0,Play with Friends Play with or against your fr...,Play with Friends Play with or against your fr...,"Salvage scrap, build defenses, and rank-up to ...",NaN,...,0,0,0,0,0,"{'Top-Down Shooter': 138, 'Rogue-lite': 132, '...",-1,-1,-1,-1


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83646 entries, 0 to 83645
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AppID                     83646 non-null  int64  
 1   name                      83643 non-null  object 
 2   release_date              83646 non-null  object 
 3   required_age              83646 non-null  int64  
 4   price                     83646 non-null  float64
 5   dlc_count                 83646 non-null  int64  
 6   detailed_description      83480 non-null  object 
 7   about_the_game            83458 non-null  object 
 8   short_description         83537 non-null  object 
 9   reviews                   10283 non-null  object 
 10  header_image              83646 non-null  object 
 11  website                   39760 non-null  object 
 12  support_url               42081 non-null  object 
 13  support_email             73067 non-null  object 
 14  window

In [20]:
data = data.dropna(subset=['name', 'detailed_description'])

Будем использовать для контентной рекомендательной системы столбцы name, genres, categories, tags, detailed_description

In [21]:
data['tags']

,tags
0,"{'FPS': 90076, 'Shooter': 64786, 'Multiplayer'..."
1,"{'Survival': 14611, 'Shooter': 12477, 'Battle ..."
2,"{'Free to Play': 59559, 'MOBA': 19966, 'Multip..."
3,"{'Open World': 31998, 'Action': 23177, 'Multip..."
4,"{'FPS': 9689, 'PvP': 9073, 'e-sports': 9026, '..."
...,...
83641,"{'Strategy': 72, 'Casual': 48, 'PvE': 47, 'Sin..."
83642,"{'Indie': 31, 'Strategy': 21, 'Puzzle': 11, 'M..."
83643,"{'Puzzle': 54, 'Colorful': 46, 'Relaxing': 44,..."
83644,"{'Top-Down Shooter': 138, 'Rogue-lite': 132, '..."


In [22]:
def tags_dict_to_list(x):
  tag_dict=eval(x)
  if type(tag_dict)!=type(list()):
    max_count = sum(tag_dict.values())
  else:
    return ["no_tags"]

  list_tags=[f"{tag}_{count / max_count}"
                      for tag, count in tag_dict.items()]
  return list_tags

In [23]:
import re

In [24]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text.split()

In [25]:
data["tags_processed"] = data["tags"].apply(tags_dict_to_list)

<ipython-input-25-a1bab2f574b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["tags_processed"] = data["tags"].apply(tags_dict_to_list)


In [26]:
def to_low(x):
  res=[i.lower() for i in eval(x)]
  return res

In [27]:
data['genres_processed']= data['genres'].apply(to_low)

<ipython-input-27-fb939a53a1bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres_processed']= data['genres'].apply(to_low)


In [28]:
data['categories_processed']=data['categories'].apply(to_low)

<ipython-input-28-58052ef5d716>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['categories_processed']=data['categories'].apply(to_low)


In [29]:
data['description_processed']=data['detailed_description'].apply(clean_text)

<ipython-input-29-60db9c402b03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['description_processed']=data['detailed_description'].apply(clean_text)


In [30]:
data["data_to_recommend"] = data.apply(lambda row: row["tags_processed"] + row["genres_processed"] +
                           row["categories_processed"] + row["description_processed"] +
                           row["name"].lower().split(), axis=1)

<ipython-input-30-cf1af94ee18d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["data_to_recommend"] = data.apply(lambda row: row["tags_processed"] + row["genres_processed"] +


In [31]:
data["data_to_recommend"]

,data_to_recommend
0,"[FPS_0.11572299248692149, Shooter_0.0832322682..."
1,"[Survival_0.12970722440210927, Shooter_0.11076..."
2,"[Free to Play_0.30531960158096283, MOBA_0.1023..."
3,"[Open World_0.10968661945276667, Action_0.0794..."
4,"[FPS_0.07254526123481933, PvP_0.06793303283966..."
...,...
83641,"[Strategy_0.13688212927756654, Casual_0.091254..."
83642,"[Indie_0.2627118644067797, Strategy_0.17796610..."
83643,"[Puzzle_0.1125, Colorful_0.09583333333333334, ..."
83644,"[Top-Down Shooter_0.0659025787965616, Rogue-li..."


In [32]:
from gensim.models import Word2Vec
from sklearn.neighbors import NearestNeighbors

In [33]:
word2vec_model = Word2Vec(sentences=data["data_to_recommend"], vector_size=100, window=5, min_count=2, workers=4)

In [34]:
def get_game_vector(words, model, vector_size=100):
    """Средний вектор по всем словам игры."""
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(vector_size)

In [35]:
data["game_vector"] = data["data_to_recommend"].apply(lambda words: get_game_vector(words, word2vec_model))

<ipython-input-35-553cc5588285>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["game_vector"] = data["data_to_recommend"].apply(lambda words: get_game_vector(words, word2vec_model))


In [36]:
data=data.reset_index()

In [37]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')

In [38]:
game_vect=np.vstack(data['game_vector'])
knn.fit(game_vect)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [159]:
print(game_vect)

[[-0.52858686  0.02186697  0.02241427 ...  0.08097721  0.00520614
   0.59661436]
 [-0.17716111 -0.5269088  -0.01911897 ... -0.61684316 -0.4379263
   0.33845085]
 [-0.26465628 -0.00243846  0.16336511 ... -0.39174     0.00370066
   0.47326553]
 ...
 [-0.38746074  0.0927595   0.09030519 ... -0.24181136  0.25861806
   0.7624338 ]
 [-0.4554165  -0.29230487  0.48294362 ... -0.62403035 -0.13789424
   0.7499115 ]
 [-0.23370896 -0.35126927  0.35929814 ... -0.52002794 -0.05296807
   0.59124833]]


In [39]:
def find_similar_games(game_name, df, knn_model, game_vectors, top_k=10):
    # Ищем индекс игры по названию
    game_idx = df[df["name"] == game_name].index

    if len(game_idx) == 0:
        print(f"Игра '{game_name}' не найдена в базе!")
        return []

    game_idx = game_idx[0]  # Берем первый индекс, если найдено несколько

    # Получаем k+1 ближайших соседей (первая — сама игра)
    distances, indices = knn_model.kneighbors([game_vectors[game_idx]], n_neighbors=top_k+1)

    # Пропускаем саму игру (первый элемент)
    similar_games = df.iloc[indices[0][1:]]  # Берем игры с 1 по k

    print(f"Похожие игры на '{game_name}':")
    return similar_games[["name", "genres", "categories"]]  # Можно добавить больше ин

In [40]:
similar_games = find_similar_games("Portal 2", data, knn, game_vect, top_k=10)
similar_games

Похожие игры на 'Portal 2':


,name,genres,categories
3689,DG2: Defense Grid 2,"['Indie', 'Strategy']","['Single-player', 'Multi-player', 'Co-op', 'Sh..."
9833,Shadow Complex Remastered,['Action'],"['Single-player', 'Steam Achievements', 'Full ..."
24799,Slime-san: Superslime Edition,"['Action', 'Adventure', 'Indie']","['Single-player', 'Multi-player', 'PvP', 'Shar..."
7633,Super Motherload,"['Adventure', 'Casual', 'Indie', 'RPG']","['Single-player', 'Multi-player', 'Shared/Spli..."
16630,Lode Runner Legacy,"['Action', 'Casual']","['Single-player', 'Multi-player', 'Co-op', 'Sh..."
19665,Q.U.B.E. 10th Anniversary,"['Action', 'Adventure', 'Indie', 'Simulation']","['Single-player', 'Steam Achievements', 'Full ..."
32771,Sentinels of Earth-Prime,"['Indie', 'Strategy']","['Single-player', 'Multi-player', 'Co-op', 'On..."
4157,Project Arrhythmia,"['Action', 'Adventure', 'Indie', 'Strategy', '...","['Single-player', 'Multi-player', 'Co-op', 'Sh..."
36538,DROD: The Second Sky,"['Adventure', 'Indie', 'Strategy']","['Single-player', 'Steam Achievements', 'Capti..."
619,Bastion,"['Action', 'Indie', 'RPG']","['Single-player', 'Steam Achievements', 'Full ..."


Нейросеть для определения схожести игр

In [90]:
from keras.models import Model
from keras.layers import Input, Dense, Lambda
import keras.backend as K
from tensorflow.keras.losses import CosineSimilarity
import tensorflow as tf

In [42]:
input_a = Input(shape=(100,))
input_b = Input(shape=(100,))

In [43]:
shared_layer = Dense(128, activation='relu')

In [44]:
encoded_a = shared_layer(input_a)
encoded_b = shared_layer(input_b)

In [91]:
distance = Lambda(lambda x: tf.expand_dims(-CosineSimilarity(axis=-1)(x[0], x[1]), axis=-1), output_shape=(1,))([encoded_a, encoded_b])

In [92]:
model = Model(inputs=[input_a, input_b], outputs=distance)

In [93]:
model.compile(loss='binary_crossentropy', optimizer='adam')

Поиск пар и определение их похожести

In [56]:
import random

In [57]:
num_pairs = 10000  # Число случайных пар, которые будем генерировать
games = data.to_dict(orient="records")

pairs = []
labels = []

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
def get_similarity(game_a, game_b):
    return cosine_similarity([game_a['game_vector']], [game_b['game_vector']])[0][0]

In [62]:
for _ in range(num_pairs):
    game_a, game_b = random.sample(games, 2)  # Случайно выбираем две игры
    similarity = get_similarity(game_a, game_b)

    pairs.append((game_a['game_vector'], game_b['game_vector']))
    if similarity > 0.8:
        labels.append(1)
    else:
        labels.append(0)

In [94]:
game_pairs = np.array(pairs)
y_labels = np.array(labels).reshape(-1,1)

In [202]:
model.fit([game_pairs[:, 0], game_pairs[:, 1]], y_labels, batch_size=1, epochs=10)

Epoch 1/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - loss: 0.1358
Epoch 2/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - loss: 0.1216
Epoch 3/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 43s 3ms/step - loss: 0.1185
Epoch 4/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 43s 3ms/step - loss: 0.1200
Epoch 5/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step - loss: 0.1151
Epoch 6/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - loss: 0.1096
Epoch 7/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 0.1077
Epoch 8/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - loss: 0.1104
Epoch 9/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - loss: 0.1015
Epoch 10/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - loss: 0.1021


In [207]:
def rec_games(game_name, data, model, top_n=10):
  selected_game= data[data['name'].str.lower() == game_name.lower()]
  game_vector = selected_game['game_vector'].values[0]
  first=np.array([game_vector for i in range(len(data))])
  second=[]

  data2=data.copy()
  for i,r in data.iterrows():
    vec=r['game_vector']
    second.append(vec)
  second=np.array(second)

  dist=model.predict([first,second], batch_size=1)
  data2['similary']=dist

  recommendations = data2[data2['name'].str.lower() != game_name.lower()]\
                      .sort_values(by='similary', ascending=True)\
                      .head(top_n)
  print(f"Похожие игры на '{game_name}':")
  return recommendations[["name", "genres", "categories"]]


In [208]:
rec_games('Portal 2', data, model)

83479/83479 ━━━━━━━━━━━━━━━━━━━━ 148s 2ms/step
Похожие игры на 'Portal 2':


,name,genres,categories
73517,深渊,"['Adventure', 'Indie', 'RPG', 'Strategy']","['Single-player', 'Family Sharing']"
20602,Gobernators (Parodia política peruana),"['Action', 'Free to Play', 'Indie']","['Single-player', 'Steam Achievements', 'Parti..."
45975,唱舞全明星,"['Action', 'Casual', 'RPG', 'Simulation', 'Fre...","['Multi-player', 'MMO', 'PvP', 'Online PvP', '..."
33404,叙事曲：难忘的回忆 / Ballade: with Memories,"['Adventure', 'Indie', 'RPG']","['Single-player', 'Steam Achievements', 'Steam..."
50335,沈める楽園,"['Adventure', 'Indie']","['Single-player', 'Family Sharing']"
50339,橙天の銀翼 ～スカイランドの魔女と巫女～,"['Indie', 'RPG']","['Single-player', 'Family Sharing']"
42007,Gigantic,"['Action', 'Free to Play', 'Strategy']","['Multi-player', 'Online PvP', 'Steam Trading ..."
75670,赋灵牌,"['Casual', 'Indie', 'RPG', 'Strategy']","['Single-player', 'Multi-player', 'Co-op', 'On..."
33367,Hold 10 Seconds,['Indie'],"['Single-player', 'Steam Achievements', 'Full ..."
20658,Neon Hardcorps,"['Action', 'Indie']","['Single-player', 'Steam Trading Cards', 'Fami..."


In [ ]:
print(len(data))

In [210]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=4fbb351224d83d86550c5e11a875534949bf017669efb445e5643785d45b0b74
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [211]:
from langdetect import detect

In [219]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [220]:
df_filtered = data[data['detailed_description'].apply(is_english)]
df_filtered

,index,AppID,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,...,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent,tags_processed,genres_processed,categories_processed,description_processed,data_to_recommend,game_vector
0,0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",...,87,8071426,80,56072,"[FPS_0.11572299248692149, Shooter_0.0832322682...","[action, free to play]","[multi-player, cross-platform multiplayer, ste...","[for, over, two, decades, counterstrike, has, ...","[FPS_0.11572299248692149, Shooter_0.0832322682...","[-0.52858686, 0.021866973, 0.022414267, 1.1040..."
1,1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,...,58,2365012,68,19517,"[Survival_0.12970722440210927, Shooter_0.11076...","[action, adventure, massively multiplayer, fre...","[multi-player, pvp, online pvp, stats, remote ...","[land, loot, survive, play, pubg, battleground...","[Survival_0.12970722440210927, Shooter_0.11076...","[-0.17716111, -0.5269088, -0.01911897, 1.11918..."
2,2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",...,81,2247365,71,23832,"[Free to Play_0.30531960158096283, MOBA_0.1023...","[action, strategy, free to play]","[multi-player, co-op, steam trading cards, ste...","[the, mostplayed, game, on, steam, every, day,...","[Free to Play_0.30531960158096283, MOBA_0.1023...","[-0.26465628, -0.002438463, 0.16336511, 0.7114..."
3,3,271590,Grand Theft Auto V,2015-04-13,17,0.00,1,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,...,87,1643791,92,16181,"[Open World_0.10968661945276667, Action_0.0794...","[action, adventure]","[single-player, multi-player, pvp, online pvp,...","[when, a, young, street, hustler, a, retired, ...","[Open World_0.10968661945276667, Action_0.0794...","[-0.47039014, -0.14481038, 0.25921765, 0.83248..."
4,4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,19.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",...,85,1091695,77,10566,"[FPS_0.07254526123481933, PvP_0.06793303283966...",[action],"[single-player, multi-player, pvp, online pvp,...","[edition, comparison, ultimate, edition, the, ...","[FPS_0.07254526123481933, PvP_0.06793303283966...","[-0.46374303, -0.21428649, 0.36305198, 1.04505..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83474,83641,1745960,Circuit Breaker,2021-10-21,0,4.99,1,Circuit Breaker is a turn-based strategy game ...,Circuit Breaker is a turn-based strategy game ...,"In the cyber-future of 2009, the most popular ...",...,-1,-1,-1,-1,"[Strategy_0.13688212927756654, Casual_0.091254...","[casual, indie, strategy]","[single-player, family sharing]","[circuit, breaker, is, a, turnbased, strategy,...","[Strategy_0.13688212927756654, Casual_0.091254...","[-0.39826664, -0.26441666, 0.4468738, 0.861050..."
83475,83642,1888920,Chess vs Chat,2022-08-09,0,3.99,0,Chess Vs Chat allows streamers to host chess m...,Chess Vs Chat allows streamers to host chess m...,Allow your chatters to actively participate an...,...,-1,-1,-1,-1,"[Indie_0.2627118644067797, Strategy_0.17796610...","[indie, strategy]","[single-player, multi-player, pvp, online pvp,...","[chess, vs, chat, allows, streamers, to, host,...","[Indie_0.2627118644067797, Strategy_0.17796610...","[-0.07868504, -0.21592598, 0.60900676, 0.74482..."
83476,83643,2166420,Orbtangle,2022-10-28,0,1.99

In [221]:
model2 = Model(inputs=[input_a, input_b], outputs=distance)

In [222]:
model2.compile(loss='binary_crossentropy', optimizer='adam')

In [229]:
games2 = df_filtered.to_dict(orient="records")

pairs2 = []
labels2 = []

In [230]:
for _ in range(num_pairs):
    game_a, game_b = random.sample(games2, 2)  # Случайно выбираем две игры
    similarity = get_similarity(game_a, game_b)

    pairs2.append((game_a['game_vector'], game_b['game_vector']))
    if similarity > 0.9:
        labels2.append(1)
    else:
        labels2.append(0)

In [231]:
game_pairs2 = np.array(pairs2)
y_labels2 = np.array(labels2).reshape(-1,1)

In [232]:
print(len(y_labels2))

10000


In [233]:
model2.fit([game_pairs2[:, 0], game_pairs2[:, 1]], y_labels2, batch_size=1, epochs=10)

Epoch 1/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - loss: 0.5092
Epoch 2/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - loss: 0.3645
Epoch 3/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.3416
Epoch 4/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - loss: 0.3425
Epoch 5/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.3423
Epoch 6/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 0.3282
Epoch 7/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - loss: 0.3263
Epoch 8/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: 0.3243
Epoch 9/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: 0.3154
Epoch 10/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - loss: 0.3166


In [234]:
rec_games('Portal 2', df_filtered, model2)

81868/81868 ━━━━━━━━━━━━━━━━━━━━ 146s 2ms/step
Похожие игры на 'Portal 2':


,name,genres,categories
45739,Paralax Vr Aim Trainer,"['Action', 'Casual', 'Indie', 'Early Access']","['Single-player', 'Tracked Controller Support'..."
27894,Bike Dash Excite!,"['Action', 'Casual']",['Single-player']
8092,Desktop Farm,"['Casual', 'Indie', 'Simulation', 'Strategy']","['Single-player', 'Steam Achievements', 'Steam..."
70774,Rifle Dodger,"['Indie', 'Early Access']","['Single-player', 'Tracked Controller Support'..."
45733,Psychopath Hunt Chapter two,"['Action', 'Adventure']","['Single-player', 'Multi-player', 'Co-op', 'On..."
27911,Kursor,"['Action', 'Casual', 'Indie']","['Single-player', 'Steam Achievements', 'Famil..."
45665,Adopted Passion Realize Your Dream,"['Adventure', 'Casual', 'Indie', 'Racing', 'Si...","['Single-player', 'Family Sharing']"
45635,Yasai Ninja,"['Action', 'Adventure', 'Indie']","['Single-player', 'Multi-player', 'Shared/Spli..."
77141,Satellite engineer,"['Casual', 'Indie', 'Free to Play']","['Single-player', 'Steam Achievements']"
54470,Cell HD: emergence,"['Action', 'Indie', 'Simulation']","['Single-player', 'Steam Achievements', 'Parti..."
